In [2]:
!pip install oracledb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 34.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
import oracledb

username = 'manuela'
password = 'teste2025'
dsn = 'localhost:1522/XEPDB1'

conexao = oracledb.connect(user=username, password=password, dsn=dsn)
cursor = conexao.cursor()


OperationalError: DPY-6005: cannot connect to database (CONNECTION_ID=DkAUU8B8PIb5ls3XhGaNxg==).
[Errno 99] Cannot assign requested address

RECEITA RECORRENTE MENSAL CLIENTES

In [ ]:
mrr = pd.read_csv(r'C:\Users\manue\faculdade\Challenge_TOTVS_2025_MassaDados_v1\mrr.csv',sep=';')

cursor.execute("""
    CREATE TABLE MRR_CLIENTES (
        CLIENTE VARCHAR2(20),
        MRR_12M NUMBER
    )
""")
dados = [tuple(x) for x in mrr.to_numpy()]

# Inserção em lote
cursor.executemany("""
    INSERT INTO MRR_CLIENTES (CLIENTE, MRR_12M)
    VALUES (:1, :2)
""", dados)

# Commit e fechamento
conexao.commit()
cursor.close()
conexao.close()

print("Dados inseridos com sucesso.")


CLIENTES DESDE

In [ ]:
clientes_desde = pd.read_csv(r'C:\Users\manue\faculdade\Challenge_TOTVS_2025_MassaDados_v1\clientes_desde.csv', sep=';')
clientes_desde['CLIENTE_DESDE'] = pd.to_datetime(clientes_desde['CLIENTE_DESDE'])

cursor.execute("""
    CREATE TABLE CLIENTES_DESDE (
        CLIENTE VARCHAR2(20),
        CLIENTE_DESDE DATE
    )
""")
# Preparar os dados como tuplas (converter datetime para date)
dados = [(row['CLIENTE'], row['CLIENTE_DESDE'].date()) for _, row in clientes_desde.iterrows()]

# Inserir os dados
cursor.executemany("""
    INSERT INTO CLIENTES_DESDE (CLIENTE, CLIENTE_DESDE)
    VALUES (:1, :2)
""", dados)

conexao.commit()
cursor.close()
conexao.close()


HISTORICO CLIENTES

In [ ]:
historico = pd.read_csv(r'C:\Users\manue\faculdade\Challenge_TOTVS_2025_MassaDados_v1\historico.csv',sep=';')

def limpar_valor(v):
    if pd.isna(v) or v == '-' or str(v).strip() == '':
        return None
    return float(str(v).replace('.', '').replace(',', '.'))

colunas_float = [
    'VL_PCT_DESC_TEMP', 'VL_PCT_DESCONTO', 'PRC_UNITARIO', 'VL_DESCONTO_TEMPORARIO',
    'VL_TOTAL', 'VL_FULL', 'VL_DESCONTO'
]
for col in colunas_float:
    historico[col] = historico[col].apply(limpar_valor)

# Também tratar MESES_BONIF e QTD
historico['MESES_BONIF'] = historico['MESES_BONIF'].apply(limpar_valor)
historico['QTD'] = historico['QTD'].apply(limpar_valor)
historico['DT_UPLOAD'] = pd.to_datetime(historico['DT_UPLOAD'], dayfirst=True, errors='coerce')


In [ ]:
cursor.execute("""
    BEGIN
        EXECUTE IMMEDIATE 'DROP TABLE HISTORICO';
    EXCEPTION
        WHEN OTHERS THEN
            IF SQLCODE != -942 THEN
                RAISE;
            END IF;
    END;
""")

cursor.execute("""
    CREATE TABLE HISTORICO (
        NR_PROPOSTA            VARCHAR2(20),
        ITEM_PROPOSTA          NUMBER,
        DT_UPLOAD              DATE,
        HOSPEDAGEM             VARCHAR2(20),
        CD_CLI                 VARCHAR2(20),
        FAT_FAIXA              VARCHAR2(50),
        CD_PROD                VARCHAR2(30),
        QTD                    NUMBER(10, 2),
        MESES_BONIF            NUMBER(10, 2),
        VL_PCT_DESC_TEMP       NUMBER(10, 2),
        VL_PCT_DESCONTO        NUMBER(10, 2),
        PRC_UNITARIO           NUMBER(15, 2),
        VL_DESCONTO_TEMPORARIO NUMBER(15, 2),
        VL_TOTAL               NUMBER(15, 2),
        VL_FULL                NUMBER(15, 2),
        VL_DESCONTO            NUMBER(15, 2)
    )
""")

dados = [
    (
        row['NR_PROPOSTA'],
        row['ITEM_PROPOSTA'],
        row['DT_UPLOAD'].date() if pd.notnull(row['DT_UPLOAD']) else None,
        row['HOSPEDAGEM'],
        row['CD_CLI'],
        row['FAT_FAIXA'],
        row['CD_PROD'],
        row['QTD'],
        row['MESES_BONIF'],
        row['VL_PCT_DESC_TEMP'],
        row['VL_PCT_DESCONTO'],
        row['PRC_UNITARIO'],
        row['VL_DESCONTO_TEMPORARIO'],
        row['VL_TOTAL'],
        row['VL_FULL'],
        row['VL_DESCONTO']
    )
    for _, row in historico.iterrows()
]

# === 7. Inserção dos dados ===
cursor.executemany("""
    INSERT INTO HISTORICO (
        NR_PROPOSTA, ITEM_PROPOSTA, DT_UPLOAD, HOSPEDAGEM, CD_CLI,
        FAT_FAIXA, CD_PROD, QTD, MESES_BONIF,
        VL_PCT_DESC_TEMP, VL_PCT_DESCONTO, PRC_UNITARIO,
        VL_DESCONTO_TEMPORARIO, VL_TOTAL, VL_FULL, VL_DESCONTO
    )
    VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, :16)
""", dados)

conexao.commit()
cursor.close()
conexao.close()


CONTRATAÇÕES ULTIMOS 12 MESES

In [ ]:
contratos_12m = pd.read_csv(r'C:\Users\manue\faculdade\Challenge_TOTVS_2025_MassaDados_v1\contratacoes_ultimos_12_meses.csv', sep=';')

contratos_12m['QTD_CONTRATACOES_12M'] = pd.to_numeric(contratos_12m['QTD_CONTRATACOES_12M'], errors='coerce')
contratos_12m['VLR_CONTRATACOES_12M'] = contratos_12m['VLR_CONTRATACOES_12M'].apply(limpar_valor)


In [ ]:
cursor.execute("""
    BEGIN
        EXECUTE IMMEDIATE 'DROP TABLE CONTRATOS_12M';
    EXCEPTION
        WHEN OTHERS THEN
            IF SQLCODE != -942 THEN
                RAISE;
            END IF;
    END;
""")

cursor.execute("""
    CREATE TABLE CONTRATOS_12M (
        CD_CLIENTE             VARCHAR2(20),
        QTD_CONTRATACOES_12M   NUMBER,
        VLR_CONTRATACOES_12M   NUMBER(15, 2)
    )
""")

dados = [
    (
        row['CD_CLIENTE'],
        row['QTD_CONTRATACOES_12M'],
        row['VLR_CONTRATACOES_12M']
    )
    for _, row in contratos_12m.iterrows()
]

cursor.executemany("""
    INSERT INTO CONTRATOS_12M (
        CD_CLIENTE, QTD_CONTRATACOES_12M, VLR_CONTRATACOES_12M
    ) VALUES (:1, :2, :3)
""", dados)

conexao.commit()
cursor.close()
conexao.close()

DADOS_CLIENTES

In [ ]:
dados_clientes = pd.read_csv(r'C:\Users\manue\faculdade\Challenge_TOTVS_2025_MassaDados_v1\dados_clientes.csv', sep=';')

from datetime import datetime

dados_clientes['VL_TOTAL_CONTRATO'] = dados_clientes['VL_TOTAL_CONTRATO'].apply(limpar_valor)

# DT_ASSINATURA_CONTRATO como datetime
dados_clientes['DT_ASSINATURA_CONTRATO'] = pd.to_datetime(
    dados_clientes['DT_ASSINATURA_CONTRATO'],
    dayfirst=True,
    errors='coerce'
)

# PAIS como número
dados_clientes['PAIS'] = pd.to_numeric(dados_clientes['PAIS'], errors='coerce')

In [ ]:
cursor.execute("""
    BEGIN
        EXECUTE IMMEDIATE 'DROP TABLE DADOS_CLIENTES';
    EXCEPTION
        WHEN OTHERS THEN
            IF SQLCODE != -942 THEN
                RAISE;
            END IF;
    END;
""")

cursor.execute("""
    CREATE TABLE DADOS_CLIENTES (
        CD_CLIENTE              VARCHAR2(20),
        DS_PROD                 VARCHAR2(100),
        DS_LIN_REC              VARCHAR2(100),
        CIDADE                  VARCHAR2(100),
        DS_CNAE                 VARCHAR2(200),
        DS_SEGMENTO             VARCHAR2(50),
        DS_SUBSEGMENTO          VARCHAR2(50),
        FAT_FAIXA               VARCHAR2(50),
        MARCA_TOTVS             VARCHAR2(50),
        MODAL_COMERC            VARCHAR2(50),
        PAIS                    NUMBER,
        PERIODICIDADE           VARCHAR2(20),
        SITUACAO_CONTRATO       VARCHAR2(20),
        UF                      VARCHAR2(2),
        VL_TOTAL_CONTRATO       NUMBER(15, 2),
        DT_ASSINATURA_CONTRATO  DATE
    )
""")


In [ ]:
# Forçar conversão segura para VARCHAR2
colunas_varchar = [
    'CD_CLIENTE', 'DS_PROD', 'DS_LIN_REC', 'CIDADE', 'DS_CNAE',
    'DS_SEGMENTO', 'DS_SUBSEGMENTO', 'FAT_FAIXA', 'MARCA_TOTVS',
    'MODAL_COMERC', 'PERIODICIDADE', 'SITUACAO_CONTRATO', 'UF'
]
for col in colunas_varchar:
    dados_clientes[col] = dados_clientes[col].astype(str)

dados = [
    (
        row['CD_CLIENTE'],
        row['DS_PROD'],
        row['DS_LIN_REC'],
        row['CIDADE'],
        row['DS_CNAE'],
        row['DS_SEGMENTO'],
        row['DS_SUBSEGMENTO'],
        row['FAT_FAIXA'],
        row['MARCA_TOTVS'],
        row['MODAL_COMERC'],
        row['PAIS'],
        row['PERIODICIDADE'],
        row['SITUACAO_CONTRATO'],
        row['UF'],
        row['VL_TOTAL_CONTRATO'],
        row['DT_ASSINATURA_CONTRATO'].date() if isinstance(row['DT_ASSINATURA_CONTRATO'], pd.Timestamp) else None
    )
    for _, row in dados_clientes.iterrows()
]
cursor.executemany("""
    INSERT INTO DADOS_CLIENTES (
        CD_CLIENTE, DS_PROD, DS_LIN_REC, CIDADE, DS_CNAE,
        DS_SEGMENTO, DS_SUBSEGMENTO, FAT_FAIXA, MARCA_TOTVS, MODAL_COMERC,
        PAIS, PERIODICIDADE, SITUACAO_CONTRATO, UF,
        VL_TOTAL_CONTRATO, DT_ASSINATURA_CONTRATO
    ) VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, :16)
""", dados)
conexao.commit()
cursor.close()
conexao.close()